# Implementation of method MOMENT on inverter data
presented by Goswami et al.


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import torch

In [2]:
def getSMAData():
    path_errors = './errortimestamps.csv'
    path_features = './featureslist.csv'
    df_errors = pd.read_csv(path_errors)
    df_features = pd.read_csv(path_features)
    total_features = list(df_features[(df_features['True/False'] == 'TRUE') | (df_features['True/False'] == 'NotAvailable')]['Features'])
    tot_feat = pd.read_csv('./totfeat.csv')
    tot_feat = list(tot_feat['feat'])
    xtrain_dfs = {}
    xtest_dfs = {}
    ytrain_dfs = {}
    ytest_dfs = {}
    for inverter in range(19):
        inv = 'inv_'+str(inverter)
        tot_feat.append('ErrBits')
        df_inv_0 = pd.read_csv('./downsampledata/'+str(inverter)+'.csv')
        df_inv_0['Timestamp'] = pd.to_datetime(df_inv_0['Timestamp'])
        df_inv_0.sort_values(by='Timestamp', inplace=True)
        df_inv_0 = df_inv_0.set_index(df_inv_0['Timestamp'])
        df_inv_0.shape


        format = '%Y-%m-%d %H:%M:%S'
        error_date = datetime.strptime(df_errors[(df_errors['Inverter'] == inverter) & (df_errors['had_failure'] == True)]['failure_time'].values[0][0:19], format)
        start_date = error_date-timedelta(days= 60)
        end_date = error_date+timedelta(days= 1*30)
        split_date = error_date-timedelta(days= 1*30)

        df_inv_0 = df_inv_0[(df_inv_0.index > start_date) & (df_inv_0.index < end_date)]
        df_inv_0 = pd.get_dummies(df_inv_0)

        for feat in tot_feat:
            if feat not in df_inv_0.columns:
                df_inv_0[feat] = 0
        df_inv_0 = df_inv_0[tot_feat]
        df_inv_0 = df_inv_0.dropna()
        df_target = df_inv_0[['ErrBits']]
        tot_feat.remove('ErrBits')
        df_inv_0 = df_inv_0[tot_feat]

        xtrain, xtest = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)], df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)]
        ytrain, ytest = df_target[(df_target.index >= start_date) & (df_target.index< split_date)], df_target[(df_target.index >= split_date) & (df_target.index< end_date)]
        print(xtrain.shape, xtest.shape)
        xtrain_dfs[inv] = xtrain
        xtest_dfs[inv] = xtest
        ytrain_dfs[inv] = ytrain
        ytest_dfs[inv] = ytest
    return xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs

In [3]:
xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs = getSMAData()

(6057, 137) (11645, 137)
(6062, 137) (11231, 137)
(6180, 137) (11258, 137)
(5490, 137) (11139, 137)
(5516, 137) (10925, 137)
(5980, 137) (12281, 137)
(5923, 137) (10865, 137)
(6180, 137) (12319, 137)
(6180, 137) (8601, 137)
(6180, 137) (12146, 137)
(6165, 137) (11853, 137)
(6176, 137) (11752, 137)
(5741, 137) (12236, 137)
(6180, 137) (12122, 137)
(6159, 137) (12339, 137)
(6162, 137) (12333, 137)
(6180, 137) (12340, 137)
(6180, 137) (12339, 137)
(6180, 137) (12310, 137)


In [4]:
import torch

# Assuming xtest_inv_0 is a pandas DataFrame
xtest_inv_0 = xtest_dfs['inv_0']
channels = [xtest_inv_0[col].tolist() for col in xtest_inv_0.columns]

# Convert the list of lists into a tensor
tensor = torch.tensor(channels)  # Shape: [n_channels, n_time_points]

# Now we want to reshape this tensor into batches of 512 time points
window_size = 512
n_channels = tensor.shape[0]  # Should be 137
n_time_points = tensor.shape[1]  # Should be 11645

# How many full windows can we extract?
n_batches = n_time_points // window_size  # Floor division to get full batches

# Truncate the tensor to the largest number of full batches (if necessary)
tensor = tensor[:, :n_batches * window_size]  # Shape: [n_channels, n_batches * 512]

# Reshape into [batch_size, n_channels, window_size]
tensor = tensor.reshape(n_channels, n_batches, window_size).permute(1, 0, 2)
# Shape: [batch_size, n_channels, window_size]

print(tensor)  # Should output: [n_batches, 137, 512]


tensor([[[786.9214, 837.3190, 855.1500,  ...,   0.0000,   0.0000,   0.0000],
         [ 58.7314,  58.5238,  58.3914,  ...,  46.9729,  47.4610,  47.8633],
         [ 60.2279,  60.0802,  59.9510,  ...,  47.6507,  48.1593,  48.6002],
         ...,
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [ 61.1295,  61.1990,  61.0895,  ...,  47.2929,  47.7819,  48.2007],
         [ 65.1100,  65.1505,  65.0505,  ...,  48.0190,  48.6012,  49.0200]],

        [[  0.0000,   0.0000,   0.0000,  ..., 198.6738, 273.6571, 370.8238],
         [ 48.1988,  48.5143,  48.8143,  ...,  48.8845,  51.2983,  59.5193],
         [ 48.8731,  49.1490,  49.4086,  ...,  49.9517,  52.5526,  60.9936],
         ...,
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [ 48.5881,  48.8205,  49.0829,  ...,  49.0157,  51.8619,  60.7955],
         [ 49.3014,  49.5729,  49.8476,  ...,  51.2229,  54.1679,  63.6860]],

        [[436.1619, 473.3214, 534.7167,  ...

In [5]:
from torch.utils.data import DataLoader, TensorDataset
batch_size = 24
dataset = TensorDataset(tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
print(dataloader)

In [6]:
from momentfm import MOMENTPipeline
model = MOMENTPipeline.from_pretrained(
    "AutonLab/MOMENT-1-large",
    model_kwargs={"task_name": "reconstruction"},  # For anomaly detection, we will load MOMENT in `reconstruction` mode
    # local_files_only=True,  # Whether or not to only look at local files (i.e., do not try to download the model).
)
model.init()

In [7]:
output = model(tensor)

C:\Users\HOELKER\.virtualenvs\moment-main\Lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [8]:
print(output)

TimeseriesOutputs(forecast=None, anomaly_scores=None, logits=None, labels=None, input_mask=tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), pretrain_mask=tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), reconstruction=tensor([[[ 6.7487e+02,  6.8271e+02,  6.7603e+02,  ...,  1.3395e+02,
           1.3205e+02,  1.0960e+02],
         [ 5.4652e+01,  5.4380e+01,  5.4033e+01,  ...,  4.6425e+01,
           4.5658e+01,  4.5141e+01],
         [ 5.8120e+01,  5.7991e+01,  5.7436e+01,  ...,  4.4308e+01,
           4.5122e+01,  4.5001e+01],
         ...,
         [ 5.5140e-07,  8.4498e-08,  2.1061e-06,  ...,  1.1250e-06,
         

In [9]:
output_rec = output.reconstruction
anomaly_scores = (output_rec - tensor) ** 2
anomaly_scores

tensor([[[1.2556e+04, 2.3905e+04, 3.2084e+04,  ..., 1.7943e+04,
          1.7436e+04, 1.2012e+04],
         [1.6644e+01, 1.7173e+01, 1.8994e+01,  ..., 3.0050e-01,
          3.2520e+00, 7.4115e+00],
         [4.4431e+00, 4.3653e+00, 6.3271e+00,  ..., 1.1172e+01,
          9.2277e+00, 1.2953e+01],
         ...,
         [3.0404e-13, 7.1399e-15, 4.4359e-12,  ..., 1.2657e-12,
          1.0303e-12, 1.6868e-13],
         [5.3253e-01, 1.5531e+00, 3.5194e+00,  ..., 1.3517e+01,
          8.1826e+00, 1.0002e+01],
         [7.5397e+00, 9.8933e+00, 1.0990e+01,  ..., 3.3567e+00,
          4.8188e+00, 4.6093e+00]],

        [[6.3566e+03, 4.2096e+03, 2.9231e+03,  ..., 6.9767e+03,
          1.0664e+04, 6.9224e+03],
         [2.1253e+01, 1.3873e+01, 7.1783e+00,  ..., 6.7486e+00,
          8.7097e+00, 9.6224e+01],
         [2.3683e+01, 1.7672e+01, 9.4523e+00,  ..., 1.5413e+02,
          2.0690e+02, 4.2029e+02],
         ...,
         [5.1913e-13, 5.2310e-13, 2.9314e-13,  ..., 7.1151e-15,
          9.581

In [34]:
# Min-Max-Normalisierung zwischen 0 und 1
min_val = tensor.min()
max_val = tensor.max()

row_to_exclude = 22

# Exclude the row using slicing
# We will concatenate slices excluding the specified row
tensor_excluded = torch.cat((tensor[:, :row_to_exclude, :], tensor[:, row_to_exclude + 1:, :]), dim=1)

# Normierter Tensor
tensor_normalized = (tensor_excluded - min_val) / (max_val - min_val)



print(tensor_normalized)

tensor([[[4.8821e-06, 5.1324e-06, 5.2209e-06,  ..., 9.7338e-07,
          9.7338e-07, 9.7338e-07],
         [1.2651e-06, 1.2641e-06, 1.2634e-06,  ..., 1.2067e-06,
          1.2091e-06, 1.2111e-06],
         [1.2725e-06, 1.2718e-06, 1.2712e-06,  ..., 1.2101e-06,
          1.2126e-06, 1.2148e-06],
         ...,
         [9.7338e-07, 9.7338e-07, 9.7338e-07,  ..., 9.7338e-07,
          9.7338e-07, 9.7338e-07],
         [1.2770e-06, 1.2774e-06, 1.2768e-06,  ..., 1.2083e-06,
          1.2107e-06, 1.2128e-06],
         [1.2968e-06, 1.2970e-06, 1.2965e-06,  ..., 1.2119e-06,
          1.2148e-06, 1.2169e-06]],

        [[9.7338e-07, 9.7338e-07, 9.7338e-07,  ..., 1.9602e-06,
          2.3326e-06, 2.8153e-06],
         [1.2128e-06, 1.2143e-06, 1.2158e-06,  ..., 1.2162e-06,
          1.2282e-06, 1.2690e-06],
         [1.2161e-06, 1.2175e-06, 1.2188e-06,  ..., 1.2215e-06,
          1.2344e-06, 1.2763e-06],
         ...,
         [9.7338e-07, 9.7338e-07, 9.7338e-07,  ..., 9.7338e-07,
          9.733

In [71]:
mask = tensor_normalized > 0.1
mask

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [

In [72]:
values_above_threshold = tensor_normalized[mask]

# Print the values that exceed the threshold
print("Values greater than the threshold:")
print(values_above_threshold)

# To print the indices as well, you can use .nonzero()
indices_above_threshold = mask.nonzero(as_tuple=False)


print("\nIndices of values greater than the threshold:")
print(indices_above_threshold)

# You can iterate over the indices and print the values with their positions
for idx in indices_above_threshold:
    print(f"Value: {tensor[tuple(idx)]}, Index: {tuple(idx)}")

Values greater than the threshold:
tensor([])

Indices of values greater than the threshold:
tensor([], size=(0, 3), dtype=torch.int64)
